In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, sigmoid_kernel, rbf_kernel, polynomial_kernel, laplacian_kernel, cosine_similarity
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from parseit.data import load_pickle, save_pickle
import pandas as pd

pd.options.display.min_rows = 650
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 20

# ranksnl stopwords (large set) was chosen; read about stopwords here (and why nltk stopwords should not be chosen):
# https://scikit-learn.org/stable/modules/feature_extraction.html#stop-words
# https://www.aclweb.org/anthology/W18-2502/
def stopwords():
    filename = os.path.join(os.getcwd(), "datasets", "ranksnl_large.csv")
    print(filename)
    stop_words = []
    with open(filename) as f:
        for line in f:
            for word in line.split(","):
                stop_words.append(word.lower().strip())
    return set(stop_words)

# There's many lists for bad words (for spam deteciton, moderation, etc). This is one of the largest I've found so far:
# https://www.freewebheaders.com/bad-words-list-and-page-moderation-words-list-for-facebook/
def badwords():
    filename = os.path.join(os.getcwd(), "datasets", "fb-bad-words.csv")
    bad_words = []
    with open(filename) as f:
        for line in f:
            for word in line.split(","):
                bad_words.append(word.lower().strip())
    return set(bad_words)

# From https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0144296:
# http://kt.ijs.si/data/Emoji_sentiment_ranking/
# Other: https://research.utwente.nl/files/5482763/sac13-senticon.pdf
# http://emojitracker.com/
def emoticons():
    filename = os.path.join(os.getcwd(), "datasets", "emoticons.csv")
    emoticons = {
        "pos": [":)", ":))", ":D", ":DD", "xD", "xDD", ":d" "=)", "=))", ":')", "=')", ":}", ":}}", ":]", ":]]", "(:", "C:", ":P"],
        "neu": [":|", ":/", ":\\", ""],
        "neg": [":(", ":((", ":(((", ":((((", ":C", ":CC", ":'C", "=(", "=((", ":'(", "='(", ":[", ":{", "):"]
    }
    with open(filename) as f:
        for line in f:
            word = line.split(",")
            smil = word[0]
            #unicodesmil = chr(int(word[1], 0))
            neg = float(word[3])
            neu = float(word[4])
            pos = float(word[5])
            if neu > pos and neu > neg:
                emoticons["neu"].append(smil)
            elif pos > neg:
                emoticons["pos"].append(smil)
                #emoticons["pos"].append(unicodesmil)
            else:
                emoticons["neg"].append(smil)
    return emoticons

# Implement our own tokenizer compatible with sklearn; we want to be able to define stopwords and vocabulary
class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`', '[removed]', '>', '*', '_', "&", "$"]
    stopwords = []
    vocab = []
    def __init__(self, stopwords=[], vocabulary=[]):
        self.wnl = WordNetLemmatizer()
        self.stopwords = stopwords
        self.vocab = vocabulary
    def __call__(self, document):
        sig_words = []
        for word, tag in pos_tag(word_tokenize(document)):
            lower_cased_tag = tag[0].lower()
            wn_tag = lower_cased_tag if lower_cased_tag in ['a', 'r', 'n', 'v'] else None
            if not wn_tag:
                lemma = word
            else:
                lemma = self.wnl.lemmatize(word, wn_tag)
            if lemma not in list(self.stopwords) + self.ignore_tokens: # and word in self.vocab:
                sig_words.append(lemma.lower())
        return sig_words

In [5]:
pickle_file_name = "99"
pickle_df = load_pickle(f"{pickle_file_name}.p")
pickle_df = pickle_df[pickle_df["body"] != "[removed]"]
pickle_df = pickle_df.reset_index()

In [6]:
df = pickle_df.copy()

swords = stopwords()
bwords = badwords()
smil = emoticons()


s_length = len(df.index)
submissions = set(df.get("submission"))
print(f"{s_length} comments in {len(submissions)} submissions")

cva = CountVectorizer(lowercase=True, tokenizer=LemmaTokenizer())
asdf = cva.fit_transform(df.get("body"))
vocabs = cva.get_feature_names()
print("Created vocabs")

cvf = CountVectorizer(stop_words=list(swords)+list(bwords), lowercase=True, tokenizer=LemmaTokenizer(stopwords=list(swords)+list(bwords)))
asdf = cvf.fit_transform(df.get("body"))
filtered_vocabs = cvf.get_feature_names()
print("Created filtered vocabs")

# Significant words count
cv = CountVectorizer(vocabulary=vocabs, stop_words=list(swords)+list(bwords), lowercase=True, tokenizer=LemmaTokenizer(stopwords=list(swords)+list(bwords)))
wc_data = cv.fit_transform(df.get("body"))
wc_df = pd.DataFrame(wc_data.sum(axis=1))
wc_df.columns = ["wc"]
print("Feature: Word count [DONE]")

# Bad words count
cv = CountVectorizer(vocabulary=bwords, stop_words=None, lowercase=True, tokenizer=LemmaTokenizer(), ngram_range=(1,2)) # finds "old bag"
bw_data = cv.fit_transform(df.get("body"))
bw_df = pd.DataFrame(bw_data.sum(axis=1))
bw_df.columns = ["bw"]
print("Feature: Bad words [DONE]")

# Stop words count
cv = CountVectorizer(vocabulary=swords, stop_words=None, lowercase=True, tokenizer=LemmaTokenizer())
sw_data = cv.fit_transform(df.get("body"))
sw_df = pd.DataFrame(sw_data.sum(axis=1))
sw_df.columns = ["sw"]
print("Feature: Stop words [DONE]")

# Positive smilies
cv = CountVectorizer(vocabulary=smil["pos"], analyzer="char", ngram_range=(1,2), stop_words=None, lowercase=False) # char + 2 ngram = ":D"
smilp_data = cv.fit_transform(df.get("body"))
smilp_df = pd.DataFrame(smilp_data.sum(axis=1))
smilp_df.columns = ["smil+"]
print("Feature: positive smiles [DONE]")

# Negative smilies count
cv = CountVectorizer(vocabulary=smil["neg"], analyzer="char", ngram_range=(1,2), stop_words=None, lowercase=False) # char + 2 ngram = ":D"
sniln_data = cv.fit_transform(df.get("body"))
smiln_df = pd.DataFrame(sniln_data.sum(axis=1))
smiln_df.columns = ["smil-"]
print("Feature: Negative smiles [DONE]")

# Neutral smilies count
cv = CountVectorizer(vocabulary=smil["neu"], analyzer="char", ngram_range=(1,2), stop_words=None, lowercase=False) # char + 2 ngram = ":D"
smile_data = cv.fit_transform(df.get("body"))
smile_df = pd.DataFrame(smile_data.sum(axis=1))
smile_df.columns = ["smil&"]
print("Feature: Neutral smiles [DONE]")


# TF-IDF cosine similarity toawrd topic
df["top-cos-sim"] = pd.Series(np.zeros(s_length), index=df.index)
for submission in submissions:
    sub_df = df[df.submission == submission][["index", "body"]]
    submission_with_comments = [submission] + list(sub_df.get("body").array)
    tfidfv = TfidfVectorizer(vocabulary=filtered_vocabs, lowercase=True, ngram_range=(1,1), smooth_idf=True, tokenizer=LemmaTokenizer(stopwords=list(swords)))
    tfidf_data = tfidfv.fit_transform(submission_with_comments)
    cosine_similarities = cosine_similarity(tfidf_data[0:1], tfidf_data[1:]).flatten()
    top_simi_df = pd.DataFrame(cosine_similarities, index=sub_df.index)
    top_simi_df.columns = ["top-cos-sim"]
    sub_df = pd.concat([sub_df, top_simi_df], axis=1)
    df.update(sub_df)
print("Feature: Cosine similarity with topic [DONE]")


# TF-IDF cosine similarity towards all documents in a submission
df["cos-sim"] = pd.Series(np.zeros(s_length), index=df.index)
for submission in submissions:
    sub_df = df[df.submission == submission][["index", "body"]]
    submission_with_comments = [" ".join(vocabs)] + list(sub_df.get("body").array)
    tfidfv = TfidfVectorizer(vocabulary=filtered_vocabs, lowercase=True, ngram_range=(1,1), smooth_idf=True, tokenizer=LemmaTokenizer(stopwords=list(swords)))
    tfidf_data = tfidfv.fit_transform(submission_with_comments)
    cosine_similarities = cosine_similarity(tfidf_data[0:1], tfidf_data[1:]).flatten()
    all_simi_df = pd.DataFrame(cosine_similarities, index=sub_df.index)
    all_simi_df.columns = ["cos-sim"]
    sub_df = pd.concat([sub_df, all_simi_df], axis=1)
    df.update(sub_df)
print("Feature: Cosine similarity with rest [DONE]")


# TF-IDF mean value (checks across all documents in a submission)
df["tfidf-mean"] = pd.Series(np.zeros(s_length), index=df.index)
for submission in submissions:
    sub_df = df[df.submission == submission][["index", "body"]]
    tfidfv = TfidfVectorizer(vocabulary=vocabs, lowercase=True, ngram_range=(1,1), smooth_idf=True, tokenizer=LemmaTokenizer(stopwords=list(swords)))
    tfidf_data = tfidfv.fit_transform(sub_df.get("body")).todense()
    means = [0]*tfidf_data.shape[0]
    for i in range(0, tfidf_data.shape[0]):
        word_count = wc_df.get("wc")[i] + bw_df.get("bw")[i] + sw_df.get("sw")[i]
        means[i] = tfidf_data[i].sum()/word_count
    tfidf_df = pd.DataFrame(means, index=sub_df.index)
    tfidf_df.columns = ["tfidf-mean"]
    sub_df = pd.concat([sub_df, tfidf_df], axis=1)
    df.update(sub_df)
print("Feature: Mean TF-IDF [DONE]")

# Update the the rest
df = pd.concat([df, wc_df, sw_df, bw_df, smilp_df, smiln_df, smile_df], axis=1)
print("Updated word counts")

save_pickle(df, f"{pickle_file_name}-with-features.p")
df



/home/halpdesk/CODE/reddit-parser/datasets/ranksnl_large.csv
43556 comments in 698 submissions


NameError: name 'filename' is not defined